# Computer Exercise 2

In [1]:
import numpy as np
import scipy as sci
import matplotlib as mp
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from numpy import pi, cos, sin, exp, sqrt
from scipy.interpolate import griddata

from mpl_toolkits.mplot3d import Axes3D
from textwrap import wrap

%matplotlib inline
%config InlineBackend.figure_format = 'pdf'

### Import required files 

In [2]:
KX = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/kx_RW.dat', 
                 sep='\s+', header=None, names=["kx"])

KY = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/ky_RW.dat', 
                 sep='\s+', header=None, names=["ky"])

KZ = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/kz_RW.dat', 
                 sep='\s+', header=None)

In [3]:
kx, ky = np.meshgrid(KX, KY)
kz = np.transpose(KZ)

dim = [np.min(kx[0,:]), np.max(kx[0,:]), np.min(ky[:,0]), np.max(ky[:,0])]
DIM = [float(dim[i]) for i in range(len(dim))]

### Pick random intergers 

In [4]:
def randomsample(x=kx, y=ky, array=kz, number=100):
    size = np.shape(array.transpose()) # Transpose may be what is messing things up
    RS = sci.sparse.bsr_matrix(size).toarray()
    rs = []
    pos= []
    count = 0
    while count < number: 
        i = np.random.randint(0, high=size[0])
        j = np.random.randint(0, high=size[1])
        # Check for repetition 
        if RS[i][j] == 0:
            RS[i][j] += kz[i][j]
            rs.append(kz[i][j])
            pos.append([kx[0][i], ky[j][0]])
            count += 1
        else:
            count += 0
    # RS = Sparce matrix with sample values at respective locations
    # rs = 1D array of the sample values
    # pos = position of the sample values
    return RS.transpose(), np.array(rs), np.array(pos)

## Let's start plotting!

### Plot the 3D surface map

In [5]:
fig = plt.figure(1, figsize=(12, 6))
SM = fig.add_subplot(111, projection='3d')

SM3D = SM.plot_surface(kx, ky, kz, rstride=10, cstride=10, cmap='inferno', antialiased=True, alpha=0.9)

SM.ticklabel_format(style='sci', axis='both', scilimits=(0,0))
SM.set_title('Rossby wave dispersion', fontsize=16)
SM.set_xlabel(r'$k_x \Lambda$', fontsize=14)
SM.set_ylabel(r'$k_y \Lambda$', fontsize=14)
SM.set_zlabel(r'$\omega$ [s$^{-1}$]', fontsize=14)

fig.colorbar(SM3D, shrink=0.75, aspect=15, format='%.1e')
plt.show()

<Figure size 864x432 with 2 Axes>

### Plot the contour

In [6]:
fig, (cp1, cp2) = plt.subplots(1, 2, figsize=(8, 5))
manual_locations = [(-0.5, 3), (-1, -3), (-1.5, 2.5), (-1.75, -2), (-2, 1.5), (-2, -1), (-2, 0.5), (-1, -0.5), (-1, 0)]

CP1 = cp1.contourf(kx, ky, kz, cmap='inferno', levels=10)
cp1.clabel(CP1, inline=False, fontsize=10, colors='w', fmt='%.1e', manual=manual_locations)
cp1.axes.set_aspect('equal')
cp1.set_xlabel(r'$k_x \Lambda$')
cp1.set_ylabel(r'$k_y \Lambda$')
cp1.set_title('Contour plot of Rossby wave dispersion')
# cp1.colorbar(CP1, shrink=1, aspect=20)

CP2 = cp2.imshow(kz, extent=DIM, origin='lower', cmap='inferno', aspect='equal', interpolation='Nearest')
cp2.axes.set_aspect('equal')
cp2.set_xlabel(r'$k_x \Lambda$')
cp2.set_ylabel(r'$k_y \Lambda$')
cp2.set_title('Heat map of Rossby wave dispersion')

fig.colorbar(CP2, shrink=1, aspect=20, format='%.1e')
fig.tight_layout()
plt.show()

<Figure size 576x360 with 3 Axes>

### Plot the sampled data

In [7]:
RS, rs, pos = randomsample()

In [8]:
fig = plt.figure(figsize=(8, 6))
# fig = plt.figure(figsize=plt.figaspect(0.5))

SD_C = fig.add_subplot(121)
sd = SD_C.contour(kx, ky, RS, linewidths=2, cmap='inferno', antialiased=False)
SD_C.axes.set_aspect('equal')

SD_Ctitle = 'Random samples from the Rossby wave dispersion'
SD_C.set_title("\n".join(wrap(SD_Ctitle, 25)))
SD_C.set_xlabel(r'$k_x \Lambda$')
SD_C.set_ylabel(r'$k_y \Lambda$')

SD_C2 = fig.add_subplot(122)
SD_C2.plot(pos[:, 0], pos[:, 1], 'w.', ms=3)
sd_c2 = SD_C2.imshow(kz, extent=DIM, origin='lower', cmap='inferno', aspect='equal', interpolation='Nearest')
SD_C2.axes.set_aspect('equal')

SD_C2title = 'Random samples from the Rossby wave dispersion'
SD_C2.set_title("\n".join(wrap(SD_C2title, 25)))
SD_C2.set_xlabel(r'$k_x \Lambda$')
SD_C2.set_ylabel(r'$k_y \Lambda$')

fig.colorbar(sd_c2, shrink=1, aspect=20, format='%.1e')
plt.show()

<Figure size 576x432 with 3 Axes>

## Interpolate the randomly sampled data

### 1) Global interpolation

In [13]:
phi = 48 * pi/180             # Latitude
Omega = 7.29 * 10**(-5)       # Angular velocity
gg = 9.8;
RR = 6370 * 10**3                   # Earth radius
hh = 4000                     # water depth


ff = 2*Omega * sin(phi)       # Coriolis parameter
bb = 2* Omega / RR * cos(phi) # beta
cc = sqrt(gg * hh)            # shallow water phase speed
LL = cc/ff                    # Rossby radius

In [24]:
kxLL = kx * LL
kyLL = ky * LL

XX = kxLL
YY = kyLL

[NX, NY] = np.shape(kz)

NSub = 100  # number of observations

ii_sub = np.random.permutation(NX)[0:NSub]
jj_sub = np.random.permutation(NY)[0:NSub]

XX_sub = np.take(XX, ii_sub)
YY_sub = np.take(YY, jj_sub)

NX_int = 100
NY_int = 200
  
XX_int_low = np.linspace(DIM[0], DIM[1], NX_int)
YY_int_low = np.linspace(DIM[2], DIM[3], NY_int)

XX_int = XX_int_low
YY_int = YY_int_low

ZZ_weight_sig1 = np.zeros((NX_int, NY_int))

In [32]:
np.shape(YY_int), np.shape(YY_sub)

((200,), (100,))

In [28]:
# Global interp method
def interp_globe(n=NX_int, search_radii=0.4):
    WW = 0
    for nx in range(1, NX_int):
        for ny in range (1, NY_int):
            rr_sq = (XX_int*nx - XX_sub)**2 + (YY_int*ny - YY_sub)**2
            ww = exp(-rr_sq / (2 * search_radii**2))
            WW += ww
            ww = ww/WW
            ZZ_weight_sig1[nx, ny] += ww * ZZ_sub
    return ZZ_weight_sig1

In [29]:
def plot_interp_globe(n=NX_int, search_radii=0.4):
    GI = interp_globe(n, search_radii)
    
    fig = plt.figure(figsize=(10, 4))

    plt.subplot(121)
    GI1 = plt.plot(XX_sub, YY_sub, 'k.')
    plt.xlabel(r'k_x $\Lambda$')
    plt.ylabel(r'k_y $\Lambda$')
    plt.title([r'$exp(-r^2/(2\sigma^2)$, $\sigma = {0} $'.format('%3.3f',sg1)])
    fontchan(FS)
    fig.colorbar(GI1)       

In [30]:
plot_interp_globe()

ValueError: operands could not be broadcast together with shapes (200,) (100,) 

### 2) Local interpolation

In [11]:
def sample_range(x=kx, y=ky, z=kz, sample_min=0, sample_max=100, steps=10,
                 D=DIM, colour='inferno', markersize=3, markercolour='w.'):
    
    RS, r, p = randomsample(number = sample_max)
    
    for i in range(sample_max, sample_min, -steps):
        rs = r[:i]
        pos = p[:i,:]
        
        RS_near = griddata(pos, rs, (x, y), method='nearest')
        RS_line = griddata(pos, rs, (x, y), method='linear')
        RS_cube = griddata(pos, rs, (x, y), method='cubic')
        
        fig = plt.figure(figsize=(15, 5))

        plt.subplot(141)
        HM = plt.imshow(z, extent=D, origin='lower', cmap=colour, aspect='equal', interpolation='Nearest')
        plt.plot(pos[:, 0], pos[:, 1], markercolour, ms=markersize)
        plt.xlabel(r'$k_x \Lambda$')
        plt.ylabel(r'$k_y \Lambda$')
        LI_O = '{0} Random samples from the Rossby wave dispersion'.format(i)
        plt.title("\n".join(wrap(LI_O, 25)))

        plt.subplot(142)
        plt.imshow(RS_near, extent=D, origin='lower', cmap=colour, aspect='equal')
        plt.xlabel(r'$k_x \Lambda$')
        plt.ylabel(r'$k_y \Lambda$')
        LI_N = 'Nearest neighbour interpolation of {0} samples from the Rossby wave dispersion'.format(i)
        plt.title("\n".join(wrap(LI_N, 30)))

        plt.subplot(143)
        plt.imshow(RS_line, extent=D, origin='lower', cmap=colour, aspect='equal')
        plt.xlabel(r'$k_x \Lambda$')
        plt.ylabel(r'$k_y \Lambda$')
        LI_L = 'Linear interpolation of {0} samples from the Rossby wave dispersion'.format(i)
        plt.title("\n".join(wrap(LI_L, 25)))

        plt.subplot(144)
        plt.imshow(RS_cube, extent=D, origin='lower', cmap=colour, aspect='equal')
        plt.xlabel(r'$k_x \Lambda$')
        plt.ylabel(r'$k_y \Lambda$')
        LI_C = 'Cubic interpolation of {0} samples from the Rossby wave dispersion'.format(i)
        plt.title("\n".join(wrap(LI_C, 25)))

        fig.colorbar(HM, shrink=1, aspect=20, format='%.1e')
        plt.show()

In [12]:
sample_range()

<Figure size 1080x360 with 5 Axes>

<Figure size 1080x360 with 5 Axes>

<Figure size 1080x360 with 5 Axes>

<Figure size 1080x360 with 5 Axes>

<Figure size 1080x360 with 5 Axes>

<Figure size 1080x360 with 5 Axes>

<Figure size 1080x360 with 5 Axes>

<Figure size 1080x360 with 5 Axes>

<Figure size 1080x360 with 5 Axes>

<Figure size 1080x360 with 5 Axes>